In [ ]:
from pathlib import Path
from tbparse import SummaryReader
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import pickle

In [ ]:
# plt.ion()
# sns.set(font_scale=1, style="white")

In [ ]:
df_file = 'dataframe.pd'
if os.path.exists(df_file):
  with open(df_file, 'rb') as f:
    df = pickle.load(f)
else:
  param_list = ["tau_mem", "scale_grad", "width_grad", "optimizer",] # "batch_size", "decoding_func", "encoding_dim", "hidden_dim", "learning_rate", "n_hidden_layers", "spike_threshold"
  dfs = []
  for root, dirs, files in os.walk(Path('lightning_logs/ssc')):
    results = None
    if 'hparams.yaml' in files:
      results = SummaryReader(root, pivot=True)
      df = results.scalars
      hp = results.hparams
      # drop a few columns that we don't use
      df = df[df.columns[~df.columns.str.endswith("weight_epoch") & ~df.columns.str.endswith("weight_step")]]
      df = df.drop(columns='hp_metric')
      # add hyperparameters to table
      df['method'] = root.split("/")[2].upper() # method name, exodus or slayer
      df['run'] = root.split("/")[-1] # we ran each experiment 3 times, so version_0, 1 or 2
      for param in param_list:
        df[param] = hp[param][0]
      dfs.append(df)
  df = pd.concat(dfs, ignore_index=True)
  df.to_pickle(df_file)

In [ ]:
sns.relplot(data=df[df["optimizer"] == "adam"], x="step", y="valid_acc", col="tau_mem", row="method", hue="scale_grad", kind="line", palette="crest")
plt.savefig("results_adam.png")

In [ ]:
sns.relplot(data=df[df["optimizer"] == "sgd"], x="step", y="valid_acc", col="tau_mem", row="method", hue="scale_grad", kind="line", palette="crest")
plt.savefig("results_sgd.png")